In [1]:
from pprint import pprint
from lxml import html
import requests
from datetime import datetime, date
import pandas as pd
import time

In [2]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

In [3]:
#yandex news parser
response = requests.get('https://yandex.ru/news/',headers=header)
ya_dom = html.fromstring(response.text)
news_blocks = ya_dom.xpath("//td[@class='stories-set__item'] | //div[@class='story__content']")
news_ya = []
for n in news_blocks:
    item = {}
    title = n.xpath(".//h2/a/text()")[0]
    a = 'https://yandex.ru' + n.xpath(".//h2/a/@href")[0]
    source = n.xpath(".//div[@class='story__date']/text()")[0][:-6]
    date_news = datetime.now().replace(hour=int(n.xpath(".//div[@class='story__date']/text()")[0][-5:][:2]), minute=int(n.xpath(".//div[@class='story__date']/text()")[0][-5:][3:]))
    item['title'] = title
    item['url'] = a
    item['source'] = source
    item['date'] = date_news
    news_ya.append(item)

In [4]:
#lenta.ru parser
response = requests.get('https://lenta.ru/',headers=header)
lenta_dom = html.fromstring(response.text)
news_blocks_top_lenta = lenta_dom.xpath("//section[contains(@class,'b-top7-for-main')]//div[contains(@class,'item')]")
news_blocks_other_lenta = lenta_dom.xpath("//section[contains(@class,'b-layout_main')]//div[contains(@class, 'b-tabloid__topic_news')] | //section[contains(@class,'b-layout_main')]//div[contains(@class, 'item') and contains(@class, 'article')]")
news_lenta = []
for n in news_blocks_top_lenta:
    item = {}
    title = n.xpath(".//a/text()")[0].replace('\xa0',' ')
    if n.xpath(".//a/@href")[0].startswith('https://'):
        a = n.xpath(".//a/@href")[0]
    else:
        a = 'https://lenta.ru' + n.xpath(".//a/@href")[0]
    source = 'lenta.ru'
    date_news = n.xpath(".//a/time/@datetime")[0].strip()
    item['title'] = title
    item['url'] = a
    item['source'] = source
    item['date'] = date_news
    news_lenta.append(item)
for n in news_blocks_other_lenta:
    item = {}
    title = '.'.join(n.xpath(".//div[contains(@class, 'titles')]//text()")).replace('\xa0', ' ')
    if n.xpath(".//h3/a/@href")[0].startswith('https://'):
        a = n.xpath(".//h3/a/@href")[0]
    else:
        a = 'https://lenta.ru' + n.xpath(".//h3/a/@href")[0]
    source = 'lenta.ru'
    date_news = n.xpath(".//span[contains(@class,'g-date item__date')]/text()")
    if not n.xpath(".//span[contains(@class,'g-date item__date')]/text()"):
        date_time = datetime.now()
    elif  'Сегодня' in n.xpath(".//span[contains(@class,'g-date item__date')]/text()"):
        date_time = datetime.now().replace(hour=int(n.xpath(".//span[contains(@class,'g-date item__date')]/span/text()")[0][:2]), minute=int(n.xpath(".//span[contains(@class,'g-date item__date')]/span/text()")[0][3:]))
    else:
        date_time = datetime.now().replace(hour=int(n.xpath(".//span[contains(@class,'g-date item__date')]/span/text()")[0][:2]), minute=int(n.xpath(".//span[contains(@class,'g-date item__date')]/span/text()")[0][3:]), day = int(n.xpath(".//span[contains(@class,'g-date item__date')]/text()")[0][:2]))
    item['title'] = title
    item['url'] = a
    item['source'] = source
    item['date'] = date_time
    news_lenta.append(item)    

In [5]:
#news.mail.ru parser
header = {'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 10_3_1 like Mac OS X) AppleWebKit/603.1.30 (KHTML, like Gecko) Version/10.0 Mobile/14E304 Safari/602.1'}
response = requests.get('https://news.mail.ru/',headers=header)
mail_dom = html.fromstring(response.text)
news_blocks_mail = mail_dom.xpath("//a[contains(@class,'item item_side_left entity')]")
news_mail = []
for n in news_blocks_mail:
    item = {}
    if n.xpath("./@href")[0].startswith('https://'):
        a = n.xpath("./@href")[0]
    else:
        a = 'https://news.mail.ru' + n.xpath("./@href")[0]
    time.sleep(1)
    response_news = requests.get(a,headers=header)
    temp_dom = html.fromstring(response_news.text)
    title = temp_dom.xpath("//h1/text() | //div[@class='cover__title']/text()")[0]
    source = temp_dom.xpath("//div[contains(@class,'article__params')]//a/text() | //a[@class='link']//span[@class='link__text']/text()")[0]
    date = temp_dom.xpath("//div[contains(@class,'article__params')]//time/@datetime")
    item['title'] = title
    item['url'] = a
    item['source'] = source
    item['date'] = ''.join(date)
    news_mail.append(item)   

### Новости Mail.ru

In [6]:
pd.DataFrame(news_mail)

,title,url,source,date
0,Украина обвинила Стрелкова в пытках и объявила...,https://news.mail.ru/politics/42209406/,РБК,2020-06-16T16:27:45+03:00
1,США разрушили нефтяной рынок союзника России,https://news.mail.ru/economics/42207481/,Lenta.Ru,2020-06-16T14:38:02+03:00
2,Виктория Боня назвала живущих на МРОТ «алкогол...,https://news.mail.ru/society/42208595/,RNS,2020-06-16T15:44:32+03:00
3,Показаны лучшие фото с высоты птичьего полета,https://news.mail.ru/society/42206354/,Новости Mail.ru,2020-06-16T13:38:29+03:00
4,Москва перешла к новому этапу снятия ограничен...,https://news.mail.ru/society/42199317/,Известия,2020-06-16T10:36:25+03:00
5,Более 50 пациентов с коронавирусом скончались ...,https://news.mail.ru/incident/42198732/,Интерфакс,2020-06-16T00:16:54+03:00
6,В Москве проведут эксперимент по снижению стои...,https://news.mail.ru/society/42205838/,РБК,2020-06-16T13:25:08+03:00
7,Строительство ЦКАД с учетом доводки подорожало...,https://news.mail.ru/society/42201074/,Интерфакс,2020-06-16T11:14:32+03:00
8,У жительницы Москвы похитили с банковских счет...,https://news.mail.ru/incident/42201605/,ТАСС,2020-06-16T09:08:35+03:00
9,В Госдуму внесен законопроект о регулировании ...,https://news.mail.ru/politics/42205558/,Коммерсантъ,2020-06-16T15:02:31+03:00


### Новости Yandex News

In [7]:
pd.DataFrame(news_ya)

,title,url,source,date
0,Полицейский и сотрудник Росгвардии погибли в п...,https://yandex.ru/news/story/Policejskij_i_sot...,Интерфакс,2020-06-16 16:35:38.540483
1,Сенатор предложил в 2021 году сократить нового...,https://yandex.ru/news/story/Senator_predlozhi...,Известия,2020-06-16 16:34:38.541483
2,В Госдуму внесли законопроект об удаленной работе,https://yandex.ru/news/story/V_Gosdumu_vnesli_...,РИА Новости,2020-06-16 16:30:38.541483
3,В ФРГ призвали отвечать на санкции США против ...,https://yandex.ru/news/story/V_FRG_prizvali_ot...,RT на русском,2020-06-16 16:32:38.541483
4,В Дагестане граждане Азербайджана начали перес...,https://yandex.ru/news/story/V_Dagestane_grazh...,RT на русском,2020-06-16 16:32:38.541483
...,...,...,...,...
60,Chevrolet вновь начал продавать в РФ бюджетные...,https://yandex.ru/news/story/Chevrolet_vnov_na...,32CARS.ru,2020-06-16 13:23:38.549484
61,В России появилась новая схема обмана при прод...,https://yandex.ru/news/story/V_Rossii_poyavila...,ПРАЙМ,2020-06-16 16:09:38.549484
62,Nissan представил новый кроссовер Nissan X-Trail,https://yandex.ru/news/story/Nissan_predstavil...,Автоновости дня,2020-06-16 12:43:38.549484
63,Российских водителей ждет ряд нововведений с 1...,https://yandex.ru/news/story/Rossijskikh_vodit...,SpeedMe,2020-06-16 16:00:38.549484


### Новости Lenta.ru

In [8]:
pd.DataFrame(news_lenta)

,title,url,source,date
0,Германия отказалась менять позицию по «Северно...,https://lenta.ru/news/2020/06/16/potok/,lenta.ru,"16:17, 16 июня 2020"
1,Порноактриса назвала главную проблему современ...,https://lenta.ru/news/2020/06/16/stoya/,lenta.ru,"16:39, 16 июня 2020"
2,Возросло число жертв смертельной перестрелки в...,https://lenta.ru/news/2020/06/16/umer/,lenta.ru,"16:35, 16 июня 2020"
3,Россиянам посоветовали пить теплую воду в жару,https://lenta.ru/news/2020/06/16/warm_water/,lenta.ru,"16:35, 16 июня 2020"
4,Эксперты объявили о лете нелегальных рейвов,https://lenta.ru/news/2020/06/16/homamrave/,lenta.ru,"16:28, 16 июня 2020"
...,...,...,...,...
71,Данные сотен тысяч любителей онлайн-знакомств ...,https://lenta.ru/news/2020/06/16/dating/,lenta.ru,2020-06-16 15:45:38.654490
72,"«Боевики заперли нас в больнице, а свои же нас...",https://lenta.ru/articles/2020/06/15/book/,lenta.ru,2020-06-15 00:02:38.655490
73,«Что? Где? Когда?» упрекнули в безграмотности,https://lenta.ru/news/2020/06/16/chto_gde_kogda/,lenta.ru,2020-06-16 14:27:38.655490
74,На одной из линий московского метро снизят сто...,https://moslenta.ru/news/na-odnoi-iz-linii-mos...,lenta.ru,2020-06-16 14:15:38.655490
